### Titanic (Compiled)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

class TitanicModel:
    def __init__(self):
        self.model = None
        self.dt = None
        self.features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'alone']
        self.target = 'survived'
        self.titanic_data = sns.load_dataset('titanic')
        self.encoder = OneHotEncoder(handle_unknown='ignore')
        self._clean()
        self._train()

    def _clean(self):
        self.titanic_data.drop(['alive', 'who', 'adult_male', 'class', 'embark_town', 'deck'], axis=1, inplace=True)
        self.titanic_data['sex'] = self.titanic_data['sex'].apply(lambda x: 1 if x == 'male' else 0)
        self.titanic_data['alone'] = self.titanic_data['alone'].apply(lambda x: 1 if x == True else 0)
        self.titanic_data.dropna(subset=['embarked'], inplace=True)
        onehot = self.encoder.fit_transform(self.titanic_data[['embarked']]).toarray()
        cols = ['embarked_' + str(val) for val in self.encoder.categories_[0]]
        onehot_df = pd.DataFrame(onehot, columns=cols)
        self.titanic_data = pd.concat([self.titanic_data, onehot_df], axis=1)
        self.features.extend(cols)
        self.titanic_data.drop(['embarked'], axis=1, inplace=True)
        self.titanic_data.dropna(inplace=True)

    def _train(self):
        X = self.titanic_data[self.features]
        y = self.titanic_data[self.target]
        self.model = LogisticRegression(max_iter=1000)
        self.model.fit(X, y)
        self.dt = DecisionTreeClassifier()
        self.dt.fit(X, y)

    def predict(self, passenger):
        passenger_df = pd.DataFrame(passenger, index=[0])
        passenger_df['sex'] = passenger_df['sex'].apply(lambda x: 1 if x == 'male' else 0)
        passenger_df['alone'] = passenger_df['alone'].apply(lambda x: 1 if x == True else 0)
        onehot = self.encoder.transform(passenger_df[['embarked']]).toarray()
        cols = ['embarked_' + str(val) for val in self.encoder.categories_[0]]
        onehot_df = pd.DataFrame(onehot, columns=cols)
        passenger_df = pd.concat([passenger_df, onehot_df], axis=1)
        passenger_df.drop(['embarked', 'name'], axis=1, inplace=True)
        die, survive = np.squeeze(self.model.predict_proba(passenger_df))
        return {'die': die, 'survive': survive}

    def feature_weights(self):
        importances = self.dt.feature_importances_
        return {feature: importance for feature, importance in zip(self.features, importances)}

def testTitanic():
    passenger = {
        'name': ['John Mortensen'],
        'pclass': [2],
        'sex': ['male'],
        'age': [64],
        'sibsp': [1],
        'parch': [1],
        'fare': [16.00],
        'embarked': ['S'],
        'alone': [False]
    }
    titanicModel = TitanicModel()
    probability = titanicModel.predict(passenger)
    print('\t death probability: {:.2%}'.format(probability.get('die')))  
    print('\t survival probability: {:.2%}'.format(probability.get('survive')))
    importances = titanicModel.feature_weights()
    for feature, importance in importances.items():
        print("\t\t", feature, f"{importance:.2%}")

if __name__ == "__main__":
    testTitanic()


	 death probability: 93.20%
	 survival probability: 6.80%
		 pclass 12.26%
		 sex 29.65%
		 age 24.84%
		 sibsp 5.06%
		 parch 1.77%
		 fare 22.89%
		 alone 0.88%
		 embarked_C 0.65%
		 embarked_Q 1.19%
		 embarked_S 0.80%


In [3]:
import base64
import io
import json
from deepface import DeepFace as dp
from PIL import Image
import cv2
import numpy as np


print("age: 34")
print("alone: true")
print("embarked: S")
print("fare: 16")
print("name: Shuban Pal")
print("parch: 2")
print("pclass: 3")
print("sex: Male")
print("sibsp: 1")

class DemographyRecognition:
    def recognize(self, base64_encoded):
        # image parsing in preparation
        b64_string = base64_encoded  # ["base64_encoded"]  # remove from dictionary

        if b64_string.startswith("data:image"):  # if headers --> remove
            b64_string = b64_string.split(",", 1)[1]

        image_data = base64.b64decode(b64_string)  # decoding

        image = Image.open(io.BytesIO(image_data))  # get the image

        image_np = np.array(image)  # convert to np array

        # analysis
        attributes = ["age", "gender"]  # which attributes
        information = dp.analyze(image_np, attributes)  # analyze

        # -------------------------------------------------------------------------------- #

        # data extraction
        data = information[0]  # get the first element
        age = data["age"]  # age extraction

        bothGenders = data["gender"]  # get both gender confidence rates
        woman = bothGenders["Woman"]  # woman confidence
        man = bothGenders["Man"]  # man confidence

        # based on the probabilities, find which is larger and return that
        gender = None
        if woman > man:
            gender = "Female"
        elif woman < man:
            gender = "Male"

        # the order is very important (must be gender THEN age)
        returnData = [gender, age]  # create a list and send it

        return returnData  # return

    def parseImage():
        pass
    
demography = DemographyRecognition.recognize(self, base64_encoded=data)

age: 34
alone: true
embarked: S
fare: 16
name: Shuban Pal
parch: 2
pclass: 3
sex: Male
sibsp: 1


NameError: name 'self' is not defined